In [1]:
%run stdPackages.ipynb
read = {'variables': ['Fundamentals', 'Load', 'Generators_Other'], 
        'variable2D': ['Generators_FuelMix'],
        'scalars': ['Scalars'],
        'maps': ['Generators_Categories']}
db = dbFromWB(os.path.join(d['data'],'0_GlobalData.xlsx'), read)
readSets(db)

# The ```mBasic.mRES``` model

In [2]:
m = mBasic.mRES(db)
m.solve()

Solution status 0: Optimization terminated successfully.


### **The model**

The model is an extension to the model [mBasic](M_mBasic.ipynb); the note "*Models for Energy Economics*" describes the extension in section 1.2.2. The extension is that there is a *minimum target for the share of total electricity production that comes from renewable sources*, or a target for Renewable Energy Shares (RES). In this model version, we identify *renewable* energy as plants that emits zero $CO2$. Thus, following the computation of emissions (also used in ```mBasic```):
$$\begin{align}
    \text{FuelConsumption}_{BFt} =& \sum_{id}\mu_{BFt}^{id}\cdot E_{id} \\ 
    \text{Emissions}_m =& \sum_{BFt}\text{FuelConsumption}_{BFt}\cdot \text{EmissionIntensity}_{BFt,m},
\end{align}$$
we define a subset of ids as ```cleanIds``` if $\sum_{BFt}\mu_{BFt}^{id}\cdot \text{EmissionIntensity}_{BFt,CO2} \leq 0$. The auxiliary property (a type of Python class method) ```cleanIds``` returns which ids we are talking about:

In [3]:
m.cleanIds

Index(['id6', 'id7', 'id8'], dtype='object', name='id')

The RES constraint can  thus be written as:
$$\begin{align}
    \sum_{id \in \text{cleanIds}} E_{id} \geq \text{Load} \cdot RESCap.
\end{align}$$
Multiply by $-1$ on both sides (this reverses the '$\geq$' to '$\leq$') to get the inequality on the form:
$$\begin{align}
    -\sum_{id \in \text{cleanIds}} E_{id} \leq - \text{Load} \cdot RESCap.
\end{align}$$


### **Augmented-form LP**

To solve the model, the algorithm in ```scipy.optimize.linprog``` requires that we specify the problem in the 'augmented'-form that specifically looks as follows:
$$\begin{align} \tag{2}
    &\min_{x}\mbox{ }c^T\cdot x \\ 
    &A_{ub}\times x \leq b_{ub} \\ 
    &A_{eq}\times x  = b_{eq} \\ 
    &l\leq x\leq u,
\end{align}$$
where 
* $c,x,l,u$ are all vectors of the same length $N$, 
* $b_{eq},b_{ub}$ are vectors of lengths $N_{eq},N_{ub}$ respectively,
* and $A_{eq}, A_{ub}$ are coefficient matrices of sizes $(N_{eq}\times N)$ and $(N_{ub} \times N)$ respectively.

Extending the model with a RES-target is relatively straightforward as it means including an inequality constraint. Specifically, the augmented form becomes (note $A_{ub}$ is transposed here):
<b id='augmentedForm'>
    
$$\begin{align}\tag{3}
c = \begin{pmatrix} mc_{id1} \\ \vdots \\ mc_{idN}\end{pmatrix}, \qquad l = \begin{pmatrix} 0 \\ \vdots \\ 0 \end{pmatrix}, \qquad u = \begin{pmatrix} q_{id1} \\ \vdots \\ q_{idN}\end{pmatrix}, \qquad b_{eq} = L, \qquad A_{eq} = \begin{pmatrix} 1 & \cdots & 1 \end{pmatrix}, \qquad b_{ub} = -\text{Load}\cdot \text{RESCap}, \qquad A_{ub} = - \begin{pmatrix} \mathbb{1}_{id1 \in \text{cleanIds}} \\  \mathbb{1}_{idN \in \text{cleanIds}} \end{pmatrix}^T,
\end{align}$$

where $\mathbb{1}_{idn\in\text{cleanIds}}$ is a dummy variable that is $1$ if $idn$ is a clean id and 0 if not.


### Adjustments to the code

Compared to ```mBasic``` the main adjustment is that there is an added ```block``` of the type ```ub```:

<img src="snippets/mBasic_RES_snippet.png" width="1300" height="400">

The specification of the constraint is in terms of syntax equivalent to the ```eq``` constraints:
* There is a constraint named ```RESCapConstraint``` ,
* with the parameter constraint $b_{ub}$ from ```-self.db['RESCap'] * self.getLoad```.
* $A_{ub}$ coefficients for this constraint are on the variable ```'Generation'``` and specifically given by $-1$, but only for the ids that are in ```self.cleanIds```. 

The conditional that only ```self.cleanIds``` should be included in the constraint is provided using the keyword ```'conditions'``` as outlined above. When supplying this keyword, the model uses a small, homemade function ```rc_pd``` that matches on index levels. Specifically, it looks at the vector ```Generation``` and checks that is defined over the index ```id```:

In [4]:
m.db['Generation']

id
id1    10.0
id2     0.0
id3    10.0
id4    30.0
id5     5.0
id6     0.0
id7    35.0
id8    10.0
dtype: float64

Then it looks at the index used in the conditional; this is also defined over ```id```:

In [5]:
m.cleanIds

Index(['id6', 'id7', 'id8'], dtype='object', name='id')

Using ```rc_pd```, we look for all overlapping domains (thus, also works with more complicated indices, also with partial overlaps):

In [6]:
rc_pd(m.db['Generation'], m.cleanIds)

id
id6     0.0
id7    35.0
id8    10.0
Name: 0, dtype: float64